## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-29-04-14-11 +0000,bbc,China holds military drills around Taiwan as w...,https://www.bbc.com/news/articles/c87l7xjp235o...
1,2025-12-29-04-13-48 +0000,startribune,"Gov. Walz calls National Guard, Minneapolis de...",https://www.startribune.com/snow-intensifies-a...
2,2025-12-29-04-00-36 +0000,nyt,Mexico Train Derailment Kills 13 People,https://www.nytimes.com/2025/12/28/world/ameri...
3,2025-12-29-04-00-00 +0000,wsj,See How a Chinese Attack on Taiwan Would Be Ja...,https://www.wsj.com/world/asia/china-taiwan-at...
4,2025-12-29-04-00-00 +0000,wsj,The Man Who Teaches Pottery Out of a Bomb Shelter,https://www.wsj.com/world/ukraine-russia-war-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,21
99,new,10
300,bardot,8
299,brigitte,8
51,zelensky,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
83,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...,66
75,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...,58
91,2025-12-28-17-32-52 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,56
161,2025-12-28-05-53-06 +0000,bbc,Zelensky says Russia 'doesn't want peace' ahea...,https://www.bbc.com/news/articles/c1dzvpg1341o...,54
104,2025-12-28-15-51-54 +0000,nypost,Trump touts ‘good and very productive’ call wi...,https://nypost.com/2025/12/28/us-news/zelensky...,52


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
83,66,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...
115,40,2025-12-28-14-17-31 +0000,cbc,"Brigitte Bardot, icon of French cinema turned ...",https://www.cbc.ca/news/entertainment/brigitte...
29,27,2025-12-29-01-26-11 +0000,nypost,Heart-pounding video captures moment cop stops...,https://nypost.com/2025/12/28/us-news/heart-po...
96,27,2025-12-28-17-00-00 +0000,wsj,The U.S. Air Force’s​ new Wedgetail ​jet progr...,https://www.wsj.com/politics/national-security...
1,23,2025-12-29-04-13-48 +0000,startribune,"Gov. Walz calls National Guard, Minneapolis de...",https://www.startribune.com/snow-intensifies-a...
33,23,2025-12-29-01-01-05 +0000,nyt,"Winter Storm Batters Minnesota, Bringing ‘Pote...",https://www.nytimes.com/2025/12/28/us/minnesot...
162,22,2025-12-28-05-33-21 +0000,nypost,"Australian cruise ship runs aground, strikes c...",https://nypost.com/2025/12/28/world-news/austr...
114,22,2025-12-28-14-17-54 +0000,nypost,Hochul throws weight behind Mamdani’s multi-bi...,https://nypost.com/2025/12/28/us-news/gov-hoch...
12,21,2025-12-29-03-03-18 +0000,nypost,"Interoceanic Train derails in southern Mexico,...",https://nypost.com/2025/12/28/world-news/inter...
37,20,2025-12-29-00-43-58 +0000,nypost,DC pipe bomb suspect Brian Cole Jr.’s possible...,https://nypost.com/2025/12/28/us-news/dc-pipe-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
